# 침수 예측 지도 시각화

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import folium

In [ ]:
# 데이터 로딩 및 준비
# 필요한 열만 선택하여 데이터프레임 필터링 (자치구 제외)
required_columns = [
    '지오메트리', '자치구', '침수 발생여부', '하천_거리', '하천면적비율',
    '건물면적_구간별_분포수', '구간별_30년_이상_건물_수', '구간별_지하_건물_수',
    '총면적', '시설녹지_개소수', '시설녹지_면적', '일반녹지_개소수',
    '일반녹지_면적', '기타녹지_개소수', '기타녹지_면적', '녹지율', '불투수면적', '불투수면적 비율',
    '3시간누적강수량', '6시간누적강수량', '9시간누적강수량', '12시간누적강수량',
    '24시간누적강수량', '36시간누적강수량', '48시간누적강수량', '침수수심'
]
merged_df = merged_df[required_columns]

# 결측치가 있는 행 제거
merged_df = merged_df.dropna()

# 지오메트리 열을 Polygon 객체로 변환하고, GeoDataFrame으로 변환
merged_df['지오메트리'] = merged_df['지오메트리'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(merged_df, geometry='지오메트리')

# CRS 설정 및 변환
if gdf.crs is None:
    gdf.set_crs(epsg=5179, inplace=True, allow_override=True)
else:
    gdf = gdf.to_crs(epsg=5179)
gdf = gdf.to_crs(epsg=4326)

# 위도와 경도 추출
gdf['위도'] = gdf.geometry.centroid.y
gdf['경도'] = gdf.geometry.centroid.x

# 자치구 컬럼만 별도로 저장
gdf_with_district = gdf[['자치구']].copy()

# 범주형 변수 인코딩 (One-Hot Encoding) (자치구 제외)
gdf_encoded = pd.get_dummies(gdf.drop(columns=['지오메트리', '위도', '경도', '자치구']), drop_first=True)

# 예측 대상 변수 설정 (예: '침수수심')
y = gdf_encoded['침수수심']
X = gdf_encoded.drop(columns=['침수수심'])

# 데이터 분할 (훈련/테스트 세트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. 데이터 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PCA 적용
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# 주성분 로딩 계산
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
num_components_to_show = 5  # 상위 5개의 주성분을 보여줌

# 주성분 로딩을 데이터프레임으로 변환
loadings_df = pd.DataFrame(loadings[:, :num_components_to_show],
                           columns=[f'PC{i+1}' for i in range(num_components_to_show)],
                           index=X.columns)

# 각 주성분(PC)별 상위 5개의 주요 변수 출력
top_features_per_pc = {}
for i in range(num_components_to_show):
    pc_name = f'PC{i+1}'
    top_features = loadings_df[pc_name].abs().sort_values(ascending=False).head(5)
    top_features_per_pc[pc_name] = top_features

# 결과 출력
for pc, features in top_features_per_pc.items():
    print(f"\n{pc}의 주요 변수들:")
    print(features)

# 전체 로딩 값을 DataFrame으로 출력하여 확인
print("\n전체 PCA 로드 행렬:")
print(loadings_df)

# PCA 고유 기여율을 백분율로 변환
explained_variance_ratio = pca.explained_variance_ratio_
explained_variance_ratio_percent = explained_variance_ratio * 100

# 고유 기여율 백분율 출력
print("고유 기여율 (Explained Variance Ratio) in %:")
for i, ratio in enumerate(explained_variance_ratio_percent, 1):
    print(f"PCA{i}: {ratio:.2f}%")

# 주요 주성분을 활용하여 데이터 차원 축소
num_components_to_use = 5  # 필요에 따라 조정
X_train_reduced = X_train_pca[:, :num_components_to_use]
X_test_reduced = X_test_pca[:, :num_components_to_use]

# 데이터 샘플링 (훈련 세트의 60% 사용)
X_train_sampled = X_train_reduced[:int(0.6 * X_train_reduced.shape[0]), :]
y_train_sampled = y_train[:int(0.6 * y_train.shape[0])]

# 모델 설정 (기본 파라미터로 설정)
models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, objective='reg:squarederror')
}

results = {}

# 모델 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train_sampled, y_train_sampled)
    y_pred = model.predict(X_test_reduced)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results[model_name] = {"MSE": mse, "R^2": r2}

    print(f"{model_name} Model")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R^2 Score: {r2:.2f}")
    print("-" * 30)

# 가장 성능이 좋은 모델 선택
best_model_name = max(results, key=lambda k: results[k]["R^2"])
best_model = models[best_model_name]

print(f"Best Model: {best_model_name}")
print(f"MSE: {results[best_model_name]['MSE']:.2f}")
print(f"R^2: {results[best_model_name]['R^2']:.2f}")

# 선택된 모델로 최종 예측 수행
best_model.fit(X_train, y_train)
best_y_pred = best_model.predict(X_test)

In [ ]:


# 대피소 데이터 불러오기
대피소 = pd.read_csv('./대피소_최종.csv')

# 자치구별로 나누어 학습 및 테스트 데이터로 샘플링하는 함수
def split_by_district(df, test_ratio=0.2):
    train_df_list = []
    test_df_list = []

    for district in df['자치구'].unique():
        district_df = df[df['자치구'] == district].copy()
        train_df, test_df = train_test_split(district_df, test_size=test_ratio, random_state=42)
        train_df_list.append(train_df)
        test_df_list.append(test_df)

    gdf_train = pd.concat(train_df_list).reset_index(drop=True)
    gdf_test = pd.concat(test_df_list).reset_index(drop=True)

    return gdf_train, gdf_test

# 전체 데이터를 자치구 비율에 따라 학습 데이터와 테스트 데이터로 나누기
gdf_train, gdf_test = split_by_district(gdf, test_ratio=0.2)

# 예측 대상 변수 설정 (예: '침수수심')
y_train = gdf_train['침수수심']
X_train = gdf_train.drop(columns=['지오메트리', '위도', '경도', '자치구', '침수수심'])
y_test = gdf_test['침수수심']
X_test = gdf_test.drop(columns=['지오메트리', '위도', '경도', '자치구', '침수수심'])

# 데이터 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PCA 적용, 컴포넌트 수를 5로 제한
pca = PCA(n_components=5)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# 모델 설정 (기본 파라미터로 설정)
models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, objective='reg:squarederror')
}

results = {}

# 모델 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train_pca, y_train)
    y_pred = model.predict(X_test_pca)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results[model_name] = {"MSE": mse, "R^2": r2}

    print(f"{model_name} Model")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R^2 Score: {r2:.2f}")
    print("-" * 30)

# 가장 성능이 좋은 모델 선택
best_model_name = max(results, key=lambda k: results[k]["R^2"])
best_model = models[best_model_name]

print(f"Best Model: {best_model_name}")
print(f"MSE: {results[best_model_name]['MSE']:.2f}")
print(f"R^2: {results[best_model_name]['R^2']:.2f}")

# 선택된 모델로 최종 예측 수행
best_model.fit(X_train_pca, y_train)
best_y_pred = best_model.predict(X_test_pca)

# 예측 결과를 포함한 GeoDataFrame 생성
gdf_test['예측수심'] = best_y_pred

# PCA 결과 추가
for i in range(pca.n_components_):
    gdf_test[f'PCA{i+1}'] = X_test_pca[:, i]

# 녹지 데이터를 불러옵니다.
녹지 = pd.read_csv('./녹지1.csv')

# 불투수율 데이터를 불러옵니다.
불투수율 = pd.read_csv('./불투수율1.csv')
# .shp 파일 경로
shp_file_path = './법정동/emd.shp'
shp_file_path2 = './시군구/sig.shp'

# shapefile을 GeoDataFrame으로 읽기 (EUC-KR 인코딩 지정)
gdf1 = gpd.read_file(shp_file_path, encoding='EUC-KR')

# 현재 좌표계를 EPSG:5174로 수동 설정
gdf1 = gdf1.set_crs(epsg=5179)
# 좌표계(CRS)를 WGS 84로 설정 (folium은 WGS 84를 사용)
gdf1 = gdf1.to_crs(epsg=4326)

gdf1 = gdf1.rename(columns={'EMD_KOR_NM': '법정동'})

# shapefile을 GeoDataFrame으로 읽기 (EUC-KR 인코딩 지정)
gdf3 = gpd.read_file(shp_file_path2, encoding='EUC-KR')

# 현재 좌표계를 EPSG:5174로 수동 설정
gdf3 = gdf3.set_crs(epsg=5179)
# 좌표계(CRS)를 WGS 84로 설정 (folium은 WGS 84를 사용)
gdf3 = gdf3.to_crs(epsg=4326)

gdf3 = gdf3.rename(columns={'SIG_KOR_NM': '자치구'})

# 자치구별 데이터 비율에 맞게 샘플링하는 함수
def sample_data_by_district(df, district_counts, num_samples=10000):
    sampled_df_list = []

    for district, ratio in district_counts.items():
        district_df = df[df['자치구'] == district]
        num_district_samples = int(ratio * num_samples)
        if len(district_df) < num_district_samples:
            sampled_df_list.append(district_df)
        else:
            sampled_df_list.append(district_df.sample(n=num_district_samples, random_state=42))

    return pd.concat(sampled_df_list)

# gdf_test에서 자치구 비율 계산
district_counts = gdf_test['자치구'].value_counts(normalize=True)

# PCA 열의 개수를 5개로 제한하고 샘플링 진행
gdf_pca_sampled_list = []
for i in range(1, 6):  # 5개의 PCA 열만 사용
    gdf_pca_sampled = sample_data_by_district(gdf_test[['위도', '경도', f'PCA{i}', '예측수심', '지오메트리', '자치구', '48시간누적강수량']], district_counts, num_samples=10000)
    gdf_pca_sampled_list.append(gdf_pca_sampled)


# 자치구 별  침수 예측 지도 시각화

In [ ]:

from branca.colormap import LinearColormap

# 기존 불투수율 데이터를 불러옵니다.
불투수율 = pd.read_csv('./불투수율1.csv')

# '지역' 열을 '자치구'와 '법정동'으로 나누기
불투수율[['자치구', '법정동']] = 불투수율['지역'].str.split(' ', n=1, expand=True)

# '불투수면 비율(%)' 열 이름을 '불투수면적 비율'로 변경
불투수율.rename(columns={'불투수면 비율(%)': '불투수면적 비율'}, inplace=True)

# 추가할 데이터를 txt 파일에서 불러오기
with open('./추가데이터.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 데이터를 리스트로 변환
추가데이터 = []
for line in lines:
    # 각 줄에서 자치구, 법정동, 불투수면적 비율 추출
    parts = line.strip().rsplit(' ', 1)
    지역 = parts[0]
    불투수면적_비율 = float(parts[1])
    추가데이터.append({"지역": 지역, "불투수면적 비율": 불투수면적_비율})

# 추가 데이터를 데이터프레임으로 변환
추가데이터_df = pd.DataFrame(추가데이터)

# '지역' 열을 '자치구'와 '법정동'으로 나누기
추가데이터_df[['자치구', '법정동']] = 추가데이터_df['지역'].str.split(' ', n=1, expand=True)

# 기존 데이터프레임과 병합
불투수율 = pd.concat([불투수율, 추가데이터_df], ignore_index=True)

# 지도 시각화 함수
def create_folium_map(df, 대피소_df, title, depth_col='예측수심'):

    for district in df['자치구'].unique():

        district_gdf = df[df['자치구'] == district]
        대피소_district = 대피소_df[대피소_df['자치구'] == district]

        # gdf3에서 자치구의 중심 좌표를 계산
        district_geom = gdf3[gdf3['자치구'] == district]['geometry'].centroid
        map_center = [district_geom.y.iloc[0], district_geom.x.iloc[0]]

        # 지도 생성
        m = folium.Map(location=map_center, zoom_start=13)

        # 침수 예상 영역 스타일링
        def style_function(feature):
            depth = feature['properties'].get(depth_col, 0)
            if depth >= 1.0:
                color = '#ff0000'  # 1m 이상 - 빨간색
            elif depth >= 0.8:
                color = '#ff4500'  # 0.8m - 다크 오렌지색
            elif depth >= 0.6:
                color = '#ffa500'  # 0.6m - 오렌지색
            elif depth >= 0.4:
                color = '#ffd700'  # 0.4m - 금색
            elif depth >= 0.2:
                color = '#ffe4b5'  # 0.2m - 연한 주황색
            else:
                color = '#ffffe0'  # 0m - 연한 노란색
            return {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.6
            }

        highlight_function = lambda x: {'weight': 3, 'fillOpacity': 0.9}
        # 60mm 기준 레이어 추가
        layer_60mm = folium.FeatureGroup(name='60mm 이하 강수량', show=False)
        gdf_60mm = district_gdf[district_gdf['48시간누적강수량'] <= 60]
        if len(gdf_60mm) != 0 :
            geojson_data_60mm = gdf_60mm.to_json()
            
            folium.GeoJson(
                geojson_data_60mm,
                style_function=style_function,
                highlight_function=highlight_function,
                tooltip=folium.GeoJsonTooltip(
                    fields=[depth_col],
                    aliases=[f'{depth_col}: '],
                    localize=True
                )
            ).add_to(layer_60mm)
            layer_60mm.add_to(m)


        # 120mm 기준 레이어 추가
        layer_120mm = folium.FeatureGroup(name='120mm 이하 강수량', show=False)
        gdf_120mm = district_gdf[district_gdf['48시간누적강수량'] <= 120]
        if len(gdf_120mm) != 0 :
            geojson_data_120mm = gdf_120mm.to_json()
            folium.GeoJson(
                geojson_data_120mm,
                style_function=style_function,
                highlight_function=highlight_function,
                tooltip=folium.GeoJsonTooltip(
                    fields=[depth_col],
                    aliases=[f'{depth_col}: '],
                    localize=True
                )
            ).add_to(layer_120mm)
            layer_120mm.add_to(m)

        # 전체 강수량 기준 레이어 추가
        layer_all = folium.FeatureGroup(name='전체 강수량', show=True)
        gdf_all = district_gdf[district_gdf['예측수심'] > 0]
        if len(gdf_all) != 0 :
            geojson_data_all = gdf_all.to_json()

            folium.GeoJson(
                geojson_data_all,
                style_function=style_function,
                highlight_function=highlight_function,
                tooltip=folium.GeoJsonTooltip(
                    fields=['예측수심'],
                    aliases=['예측수심: '],
                    localize=True
                )
            ).add_to(layer_all)
            layer_all.add_to(m)
        else : 
            print(f"{district}")
        # 대피소 레이어 추가 (긴급 및 안전 분리)
        긴급_layer = folium.FeatureGroup(name='긴급 대피소', show=False).add_to(m)
        안전_layer = folium.FeatureGroup(name='안전 대피소', show=False).add_to(m)

        for _, row in 대피소_district.iterrows():
            if row['대피소_분류_구분'] == '긴급':
                folium.Marker(
                    location=[row['위도'], row['경도']],
                    popup=f"{row['대피소_이름']} ({row['대피소_분류_구분']})",
                    icon=folium.Icon(color='red')
                ).add_to(긴급_layer)
            else:
                folium.Marker(
                    location=[row['위도'], row['경도']],
                    popup=f"{row['대피소_이름']} ({row['대피소_분류_구분']})",
                    icon=folium.Icon(color='blue')
                ).add_to(안전_layer)

        # 녹지 데이터 레이어 추가
                
        # 녹지율 범례 HTML 추가 (녹지 비율)
        녹지_legend_html = """
        <div id="녹지_legend" style="
            position: fixed;
            top: 10px; left: calc(50% - 320px); width: 300px; height: 50px; 
            background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
            ">
            <div style="text-align:center; padding: 5px; background-color: black; color: white;">
                녹지율(%)
            </div>
            <div style="height: 20px; background: linear-gradient(to right, yellow, green);"></div>
            <div style="display: flex; justify-content: space-between; padding: 0 10px;">
                <span style="color: white;">0%</span>
                <span style="color: white;">100%</span>
            </div>
        </div>
        """

        # 지도에 녹지율 범례 HTML 추가
        m.get_root().html.add_child(folium.Element(녹지_legend_html))
        녹지1 = 녹지[녹지['자치구']==district]
        녹지_layer = folium.FeatureGroup(name='녹지 비율', show=False)
        녹지_layer = folium.FeatureGroup(name='녹지 비율', show=False)
        for index, row in 녹지1.iterrows():
            dong_name = row['법정동'].strip()  # 자치구 이름, 공백 제거
            녹지율 = row['녹지비율']  # 녹지율

            # 해당 자치구에 대한 경계 데이터 필터링
            gdf_target = gdf1[gdf1['법정동'] == dong_name]

            if not gdf_target.empty:  # gdf_target이 비어있지 않을 경우에만 처리
                # 색상 계산 (녹지율에 따라 색상 변환)
                color1 = folium.LinearColormap(['yellow', 'green'], vmin=0, vmax=100)(녹지율)

                # 지도에 자치구 경계와 함께 색상 표시 및 툴팁 추가
                folium.GeoJson(
                    gdf_target.to_json(),
                    style_function=lambda feature, color=color1: {
                        'fillColor': color,
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.6,
                    },
                    tooltip=folium.GeoJsonTooltip(
                        fields=['법정동'],
                        aliases=[f'녹지율: {녹지율}%'],
                        sticky=True
                    )
                ).add_to(녹지_layer)

        녹지_layer.add_to(m)
        
        # 불투수율 범례 HTML 추가 (불투수율 범례를 녹지 범례 오른쪽에 배치)
        legend_html = """
        <div id="legend" style="
            position: fixed;
            top: 10px; left: calc(50% + 20px); width: 300px; height: 50px; 
            background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
            ">
            <div style="text-align:center; padding: 5px; background-color: black; color: white;">
                불투수율 (%)
            </div>
            <div style="height: 20px; background: linear-gradient(to right, green, red);"></div>
            <div style="display: flex; justify-content: space-between; padding: 0 10px;">
                <span style="color: white;">0%</span>
                <span style="color: white;">100%</span>
            </div>
        </div>
        """

        # 지도에 불투수율 범례 HTML 추가
        m.get_root().html.add_child(folium.Element(legend_html))
        
        # 녹지 데이터 레이어 추가
        불투수율1 = 불투수율[불투수율['자치구']==district]
        불투수율_layer = folium.FeatureGroup(name='불투수면적 비율', show=False)
        for index, row in 불투수율1.iterrows():
            dong_name = row['법정동'].strip()  # 자치구 이름, 공백 제거
            불투수비율 = row['불투수면적 비율']  # 불투수율

            # 해당 자치구에 대한 경계 데이터 필터링
            gdf_target = gdf1[gdf1['법정동'] == dong_name]

            if not gdf_target.empty:  # gdf_target이 비어있지 않을 경우에만 처리
                # 색상 계산 (불투수율에 따라 색상 변환)
                color1 = folium.LinearColormap(['green', 'red'], vmin=0, vmax=100)(불투수비율)

                # 지도에 자치구 경계와 함께 색상 표시 및 툴팁 추가
                folium.GeoJson(
                    gdf_target.to_json(),
                    style_function=lambda feature, color=color1: {
                        'fillColor': color,
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.6,
                    },
                    tooltip=folium.GeoJsonTooltip(
                        fields=['법정동'],
                        aliases=[f'불투수율: {불투수비율}%'],
                        sticky=True
                    )
                ).add_to(불투수율_layer)

        불투수율_layer.add_to(m)
        
        remaining_gdf = gdf3[gdf3['자치구'] != district]
        folium.GeoJson(
            remaining_gdf.to_json(),
            style_function=lambda x: {
                'fillColor': '#000000',
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.8
            }
        ).add_to(m)

        # 레이어 컨트롤 추가 (축소된 상태로 시작)
        folium.LayerControl(collapsed=True).add_to(m)

        # 지도 저장
        m.save(f"./{title}_침수예상지도_{district}.html")

# PCA1과 PCA2 별로 지도 생성 및 저장
create_folium_map(gdf_pca_sampled_list[0], 대피소, 'PCA1_Predicted_Depth_Map', depth_col='예측수심')

# 서울 예측 침수 지도 시각화

In [ ]:
import chardet
import json
import geopandas as gpd
import folium
import pandas as pd
from branca.colormap import LinearColormap
from IPython.display import display, HTML
# 대피소 데이터 불러오기
대피소 = pd.read_csv('./대피소_최종.csv')

gdf_pca1_sampled = gdf_pca_sampled_list[0]

# 불투수율 데이터를 불러옵니다.
녹지 = pd.read_csv('./녹지1.csv')

# 기존 불투수율 데이터를 불러옵니다.
불투수율 = pd.read_csv('./불투수율1.csv')

# '지역' 열을 '자치구'와 '법정동'으로 나누기
불투수율[['자치구', '법정동']] = 불투수율['지역'].str.split(' ', n=1, expand=True)

# '불투수면 비율(%)' 열 이름을 '불투수면적 비율'로 변경
불투수율.rename(columns={'불투수면 비율(%)': '불투수면적 비율'}, inplace=True)

# 추가할 데이터를 txt 파일에서 불러오기
with open('./추가데이터.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 데이터를 리스트로 변환
추가데이터 = []
for line in lines:
    # 각 줄에서 자치구, 법정동, 불투수면적 비율 추출
    parts = line.strip().rsplit(' ', 1)
    지역 = parts[0]
    불투수면적_비율 = float(parts[1])
    추가데이터.append({"지역": 지역, "불투수면적 비율": 불투수면적_비율})

# 추가 데이터를 데이터프레임으로 변환
추가데이터_df = pd.DataFrame(추가데이터)

# '지역' 열을 '자치구'와 '법정동'으로 나누기
추가데이터_df[['자치구', '법정동']] = 추가데이터_df['지역'].str.split(' ', n=1, expand=True)

# 기존 데이터프레임과 병합
불투수율 = pd.concat([불투수율, 추가데이터_df], ignore_index=True)

# .shp 파일 경로
shp_file_path = './법정동/emd.shp'
shp_file_path2 = './시도/ctprvn.shp'

# shapefile을 GeoDataFrame으로 읽기 (EUC-KR 인코딩 지정)
gdf1 = gpd.read_file(shp_file_path, encoding='EUC-KR')

# 현재 좌표계를 EPSG:5174로 수동 설정
gdf1 = gdf1.set_crs(epsg=5179)
# 좌표계(CRS)를 WGS 84로 설정 (folium은 WGS 84를 사용)
gdf1 = gdf1.to_crs(epsg=4326)

gdf1 = gdf1.rename(columns={'EMD_KOR_NM': '법정동'})

# shapefile을 GeoDataFrame으로 읽기 (EUC-KR 인코딩 지정)
gdf3 = gpd.read_file(shp_file_path2, encoding='EUC-KR')

# 현재 좌표계를 EPSG:5174로 수동 설정
gdf3 = gdf3.set_crs(epsg=5179)
# 좌표계(CRS)를 WGS 84로 설정 (folium은 WGS 84를 사용)
gdf3 = gdf3.to_crs(epsg=4326)

gdf3 = gdf3.rename(columns={'CTP_KOR_NM': '시단위'})

# 지도 생성
map_center = [37.5665, 126.9780]
m = folium.Map(location=map_center, zoom_start=11)

# 침수 예상 영역 스타일링 함수
def style_function(feature):
    depth = feature['properties'].get('예측수심', 0)
    if depth >= 1.0:
        color = '#ff0000'  # 1m 이상 - 빨간색
    elif depth >= 0.8:
        color = '#ff4500'  # 0.8m - 다크 오렌지색
    elif depth >= 0.6:
        color = '#ffa500'  # 0.6m - 오렌지색
    elif depth >= 0.4:
        color = '#ffd700'  # 0.4m - 금색
    elif depth >= 0.2:
        color = '#ffe4b5'  # 0.2m - 연한 주황색
    else:
        color = '#ffffe0'  # 0m - 연한 노란색
    return {
        'fillColor': color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6
    }

highlight_function = lambda x: {'weight': 3, 'fillOpacity': 0.9}

# 60mm 기준 레이어 추가
layer_60mm = folium.FeatureGroup(name='60mm 이하 강수량', show=False)
gdf_60mm = gdf_pca1_sampled[(gdf_pca1_sampled['48시간누적강수량'] <= 60) & (gdf_pca1_sampled['예측수심'] > 0)]
geojson_data_60mm = gdf_60mm.to_json()

folium.GeoJson(
    geojson_data_60mm,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['예측수심'],
        aliases=['예측수심: '],
        localize=True
    )
).add_to(layer_60mm)
layer_60mm.add_to(m)

# 120mm 기준 레이어 추가
layer_120mm = folium.FeatureGroup(name='120mm 이하 강수량', show=False)
gdf_120mm = gdf_pca1_sampled[(gdf_pca1_sampled['48시간누적강수량'] <= 120) & (gdf_pca1_sampled['예측수심'] > 0)]
geojson_data_120mm = gdf_120mm.to_json()

folium.GeoJson(
    geojson_data_120mm,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['예측수심'],
        aliases=['예측수심: '],
        localize=True
    )
).add_to(layer_120mm)
layer_120mm.add_to(m)

# 전체 강수량 기준 레이어 추가
layer_all = folium.FeatureGroup(name='전체 강수량', show=True)
gdf_all = gdf_pca1_sampled[gdf_pca1_sampled['예측수심'] > 0]
geojson_data_all = gdf_all.to_json()

folium.GeoJson(
    geojson_data_all,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['예측수심'],
        aliases=['예측수심: '],
        localize=True
    )
).add_to(layer_all)
layer_all.add_to(m)

# 대피소 레이어 추가 (긴급 및 안전 분리)
긴급_layer = folium.FeatureGroup(name='긴급 대피소', show=False).add_to(m)
안전_layer = folium.FeatureGroup(name='안전 대피소', show=False).add_to(m)

for _, row in 대피소.iterrows():
    if row['대피소_분류_구분'] == '긴급':
        folium.Marker(
            location=[row['위도'], row['경도']],
            popup=f"{row['대피소_이름']} ({row['대피소_분류_구분']})",
            icon=folium.Icon(color='red')
        ).add_to(긴급_layer)
    else:
        folium.Marker(
            location=[row['위도'], row['경도']],
            popup=f"{row['대피소_이름']} ({row['대피소_분류_구분']})",
            icon=folium.Icon(color='blue')
        ).add_to(안전_layer)


# 녹지율 범례 HTML 추가 (녹지 비율)
녹지_legend_html = """
<div id="녹지_legend" style="
    position: fixed;
    top: 10px; left: calc(50% - 320px); width: 300px; height: 50px; 
    background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
    ">
    <div style="text-align:center; padding: 5px; background-color: black; color: white;">
        녹지율(%)
    </div>
    <div style="height: 20px; background: linear-gradient(to right, yellow, green);"></div>
    <div style="display: flex; justify-content: space-between; padding: 0 10px;">
        <span style="color: white;">0%</span>
        <span style="color: white;">100%</span>
    </div>
</div>
"""

# 지도에 녹지율 범례 HTML 추가
m.get_root().html.add_child(folium.Element(녹지_legend_html))
# 녹지 데이터 레이어 추가
녹지_layer = folium.FeatureGroup(name='녹지 비율', show=False)
for index, row in 녹지.iterrows():
    dong_name = row['법정동'].strip()  # 자치구 이름, 공백 제거
    녹지율 = row['녹지비율']  # 녹지율

    # 해당 자치구에 대한 경계 데이터 필터링
    gdf_target = gdf1[gdf1['법정동'] == dong_name]

    if not gdf_target.empty:  # gdf_target이 비어있지 않을 경우에만 처리
        # 색상 계산 (녹지율에 따라 색상 변환)
        color1 = folium.LinearColormap(['yellow', 'green'], vmin=0, vmax=100)(녹지율)

        # 지도에 자치구 경계와 함께 색상 표시 및 툴팁 추가
        folium.GeoJson(
            gdf_target.to_json(),
            style_function=lambda feature, color=color1: {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.6,
            },
            tooltip=folium.GeoJsonTooltip(
                fields=['법정동'],
                aliases=[f'녹지율: {녹지율}%'],
                sticky=True
            )
        ).add_to(녹지_layer)

녹지_layer.add_to(m)


# 불투수율 범례 HTML 추가 (불투수율 범례를 녹지 범례 오른쪽에 배치)
legend_html = """
<div id="legend" style="
    position: fixed;
    top: 10px; left: calc(50% + 20px); width: 300px; height: 50px; 
    background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
    ">
    <div style="text-align:center; padding: 5px; background-color: black; color: white;">
        불투수율 (%)
    </div>
    <div style="height: 20px; background: linear-gradient(to right, green, red);"></div>
    <div style="display: flex; justify-content: space-between; padding: 0 10px;">
        <span style="color: white;">0%</span>
        <span style="color: white;">100%</span>
    </div>
</div>
"""

# 지도에 불투수율 범례 HTML 추가
m.get_root().html.add_child(folium.Element(legend_html))

# 불투수율 데이터 레이어 추가 (위에 코드를 그대로 사용)
불투수율_layer = folium.FeatureGroup(name='불투수면적 비율', show=False)
for index, row in 불투수율.iterrows():
    dong_name = row['법정동'].strip()  # 자치구 이름, 공백 제거
    불투수비율 = row['불투수면적 비율']  # 불투수율

    # 해당 자치구에 대한 경계 데이터 필터링
    gdf_target = gdf1[gdf1['법정동'] == dong_name]


    if not gdf_target.empty:  # gdf_target이 비어있지 않을 경우에만 처리
        # 색상 계산 (불투수율에 따라 색상 변환)
        color1 = folium.LinearColormap(['green', 'red'], vmin=0, vmax=100)(불투수비율)

        # 지도에 자치구 경계와 함께 색상 표시 및 툴팁 추가
        folium.GeoJson(
            gdf_target.to_json(),
            style_function=lambda feature, color=color1: {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.6,
            },
            tooltip=folium.GeoJsonTooltip(
                fields=['법정동'],
                aliases=[f'불투수율: {불투수비율}%'],
                sticky=True
            )
        ).add_to(불투수율_layer)

    불투수율_layer.add_to(m)

# 나머지 자치구 영역 회색 처리
remaining_gdf = gdf3[gdf3['시단위'] != '서울특별시']
folium.GeoJson(
    remaining_gdf.to_json(),
    style_function=lambda x: {
        'fillColor': '#000000',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.8
    }
).add_to(m)

# 레이어 컨트롤 추가 (축소된 상태로 시작)
folium.LayerControl(collapsed=True,position='topright').add_to(m)

# 범례를 추가한 지도 저장
m.save(f"./PCA1_Predicted_Depth_Map_침수예상지도.html")



# 자치구 별 info 지도 시각화 

In [ ]:
인구밀도 = pd.read_csv('인구밀도.csv',encoding='utf-8-sig')
거주인구 = pd.read_csv('행정동단위_거주인구.csv',encoding='utf-8-sig')
의료기관 = pd.read_csv('의료기관(구별).csv',encoding='utf-8-sig')
경찰 = pd.read_csv('서울시_경찰청위치데이터.csv',encoding='utf-8-sig')
노인 = pd.read_csv('서울시 고령자현황 데이터 2023.csv', encoding='cp949')
장애인 = pd.read_csv('서울시 구별 장애인 데이터.csv',encoding='cp949')
기초생활수급자 = pd.read_csv('기초생활수급자.csv',encoding='cp949')

- 인구 밀도 전처리

In [ ]:
# '지역구' 컬럼 이름을 '자치구'로 변경
# 자치구별

인구밀도.rename(columns={'지역구': '자치구'}, inplace=True)

종합_인구밀도 = 인구밀도[인구밀도['자치구'] != '종합']
종합_인구밀도 = 종합_인구밀도[종합_인구밀도['동별'] == '종합']
종합_인구밀도 = 종합_인구밀도[['자치구', '인구밀도 (명/㎢)']] ; 종합_인구밀도.head(3)


- 경찰 관련 전처리

In [ ]:
경찰 = 경찰.drop(columns=['Unnamed: 0'])
경찰 = 경찰.drop(columns=['연번'])

# '서울특별시' 다음의 자치구명을 추출하여 새로운 컬럼 '자치구'로 추가
경찰['자치구'] = 경찰['주소'].apply(lambda x: x.split()[1])

# 자치구별로 정리된 데이터 확인
경찰['자치구'].unique()


In [ ]:
# 서울시의 모든 자치구 목록
서울_자치구 = [
    '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구',
    '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구',
    '관악구', '서초구', '강남구', '송파구', '강동구'
]

# 자치구 컬럼에서 서울시 자치구 목록에 없는 경우 필터링
예외_자치구 = 경찰[~경찰['자치구'].isin(서울_자치구)]

# 예외적인 데이터 출력
print(예외_자치구[['주소', '자치구']])

In [ ]:
# 자치구를 '강서구'로 수정
경찰.loc[경찰['자치구'] == '허준로221-22', '자치구'] = '강서구'
# 자치구별로 정리된 데이터 확인
경찰['자치구'].unique()

In [ ]:
# 자치구별로 경찰서 개수 세기
경찰서 = 경찰.groupby('자치구').size().reset_index(name='경찰서수')

# 개수로 정렬 (내림차순)
경찰서 = 경찰서.sort_values(by='경찰서수', ascending=False)

# 결과 확인
경찰서.head(1)


- 병원 관련 전처리

In [ ]:
병원 = 의료기관[의료기관['자치구'] != '소계']
병원 = 병원[['자치구', '병원수']] ; 병원.head(3)

- 구난 시설 관련 전처리

In [ ]:
# 경찰서와 병원 데이터를 자치구를 기준으로 병합
구난시설 = pd.merge(경찰서, 병원, on='자치구')

# '구난시설' 열 생성: 경찰서수와 병원수의 합
구난시설['구난시설'] = 구난시설['경찰서수'] + 구난시설['병원수']
구난시설.head(3)

- 노인 관련 전처리

In [ ]:
노인.rename(columns={'구별코드': '자치구'}, inplace=True)
노인.rename(columns={'전체인구': '노인_전체인구'}, inplace=True)
노인 = 노인[노인['자치구'] != '소계']
노인 = 노인[['자치구', '노인_전체인구']] ; 노인.head(3)

- 장애인 관련 전처리

In [ ]:
장애인.rename(columns={'구별코드': '자치구'}, inplace=True)
장애인.rename(columns={'계': '장애인_전체인구'}, inplace=True)
장애인 = 장애인[장애인['자치구'] != '계']
장애인 = 장애인[['자치구', '장애인_전체인구']] ; 장애인.head(3)

- 기초생활수급자 관련 전처리

In [ ]:
# 1. '구별코드' 컬럼 이름을 '자치구'로 변경
기초생활수급자.rename(columns={'구별코드': '자치구'}, inplace=True)

# 2. '총 수급자 남'과 '총 수급자 여'의 NaN 및 무한대 값 처리 후 정수형으로 변환
기초생활수급자['총 수급자 남'] = 기초생활수급자['총 수급자 남'].replace([np.inf, -np.inf], 0).fillna(0)
기초생활수급자['총 수급자 여'] = 기초생활수급자['총 수급자 여'].replace([np.inf, -np.inf], 0).fillna(0)

# '기초생활수급자_전체인구' 계산
기초생활수급자['기초생활수급자_전체인구'] = (기초생활수급자['총 수급자 남'] + 기초생활수급자['총 수급자 여']).astype(int)

# 3. '자치구'가 '합계'와 '본청'인 행을 제외
기초생활수급자 = 기초생활수급자[~기초생활수급자['자치구'].isin(['합계', '본청'])]

# 4. 자치구별로 '기초생활수급자_전체인구' 합산
기초생활수급자 = 기초생활수급자.groupby('자치구', as_index=False).agg({
    '기초생활수급자_전체인구': 'sum'
})

# 결과 확인
기초생활수급자.head(3)


- 거주인구 관련 전처리

In [ ]:
# '행정동' 컬럼에서 자치구를 추출하여 '자치구' 컬럼에 저장
거주인구['자치구'] = 거주인구['행정동'].apply(lambda x: x.split()[0])

# '자치구' 컬럼을 첫 번째 위치로 이동
columns = ['자치구'] + [col for col in 거주인구.columns if col != '자치구']
거주인구 = 거주인구[columns]

# 자치구별로 데이터 집계
거주인구 = 거주인구.groupby('자치구').sum().reset_index()

# '자치구'가 'Unknown'인 행 제거
거주인구 = 거주인구[거주인구['자치구'] != 'Unknown']

# 자치구와 총생활인구수만 선택
거주인구 = 거주인구[['자치구', '총생활인구수']]

# 결과 확인
거주인구.head(3)

In [ ]:
# 자치구별로 총생활인구수와 노인 인구 수를 병합
social_df = pd.merge(거주인구, 노인, on='자치구', how='left')

# 노인 인구 비율 계산
social_df['노인 인구 비율 (%)'] = (social_df['노인_전체인구'] / social_df['총생활인구수']) * 100

# 소수점 둘째 자리까지 출력 포맷 설정
pd.options.display.float_format = '{:.2f}'.format

# 결과 확인
social_df[['자치구', '노인 인구 비율 (%)']].head(3)


In [ ]:
# 자치구별로 총생활인구수와 장애인 인구 수를 병합
social_df = pd.merge(거주인구, 장애인, on='자치구', how='left')

# 장애인 인구 비율 계산
social_df['장애인 인구 비율 (%)'] = (social_df['장애인_전체인구'] / social_df['총생활인구수']) * 100

# 소수점 둘째 자리까지 출력 포맷 설정
pd.options.display.float_format = '{:.3f}'.format

# 결과 확인
social_df[['자치구', '장애인 인구 비율 (%)']].head(3)

In [ ]:
# 자치구별로 총생활인구수와 기초생활수급자_전체인구를 병합
social_df = pd.merge(거주인구, 기초생활수급자, on='자치구', how='left')

# 기초생활수급자 인구 비율 계산
social_df['기초생활수급자_인구 비율 (%)'] = (social_df['기초생활수급자_전체인구'] / social_df['총생활인구수']) * 100

# 소수점 둘째 자리까지 출력 포맷 설정
pd.options.display.float_format = '{:.3f}'.format

# 결과 확인
social_df[['자치구', '기초생활수급자_인구 비율 (%)']].head(3)

In [ ]:
구전체면적 = pd.DataFrame({
    '자치구': ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구',
             '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구',
             '관악구', '서초구', '강남구', '송파구', '강동구'],
    '구전체면적': [2312.4, 998.2, 2196.7, 1642.2, 1788.9, 1459.4, 1829.9, 2487.1, 2381.9, 2077.7,
                 3575.5, 3133.1, 1782.2, 2347.4, 1736.6, 4207.7, 2009.6, 1294.6, 2461.8, 1646.0,
                 2996.6, 4695.5, 3970.4, 3310.9, 2523.6]  # 제공된 데이터
})

# 총생활인구수와 구난시설 수를 병합
social_df = pd.merge(거주인구, 구난시설, on='자치구', how='left')
social_df = pd.merge(social_df, 구전체면적, on='자치구', how='left')

# 구난시설 비율 계산 (면적 대비 구난시설 개수)
social_df['구난시설 비율 (시설/면적)'] = social_df['구난시설'] / social_df['구전체면적']

# 소수점 둘째 자리까지 포맷 설정
pd.options.display.float_format = '{:.2f}'.format

# 결과 확인
social_df[['자치구', '구난시설 비율 (시설/면적)']].head(3)

In [ ]:
# 실제 존재하는 PCA 열 목록 추출
existing_pca_columns = [col for col in sampled_gdf_test.columns if col.startswith('PCA')]

# 자치구 정보를 포함한 샘플링된 데이터프레임 생성
# 'Cluster', '자치구' 열이 포함되도록 확장
required_columns = existing_pca_columns + ['Cluster', '자치구']
sampled_gdf_test = sampled_gdf_test[required_columns]

# 각 자치구가 속한 클러스터 확인 및 클러스터별로 정렬
# '자치구', 'Cluster' 순으로 그룹화 후 'Count' 기준으로 정렬
cluster_counts = sampled_gdf_test.groupby(['Cluster', '자치구']).size().reset_index(name='Count')

# 클러스터별로 정렬
cluster_counts_sorted = cluster_counts.sort_values(by=['Cluster', 'Count'], ascending=[True, False])

# 결과 출력
cluster_counts_sorted

In [ ]:
import pandas as pd

# 자치구별로 총생활인구수와 노인 인구 수를 병합
social_df = pd.merge(거주인구, 노인, on='자치구', how='left')

# 노인 인구 비율 계산
social_df['노인 인구 비율 (%)'] = (social_df['노인_전체인구'] / social_df['총생활인구수']) * 100

# 자치구별로 총생활인구수와 장애인 인구 수를 병합
social_df = pd.merge(social_df, 장애인, on='자치구', how='left')

# 장애인 인구 비율 계산
social_df['장애인 인구 비율 (%)'] = (social_df['장애인_전체인구'] / social_df['총생활인구수']) * 100

# 자치구별로 총생활인구수와 기초생활수급자_전체인구를 병합
social_df = pd.merge(social_df, 기초생활수급자, on='자치구', how='left')

# 기초생활수급자 인구 비율 계산
social_df['기초생활수급자_인구 비율 (%)'] = (social_df['기초생활수급자_전체인구'] / social_df['총생활인구수']) * 100

# 구전체면적 데이터를 병합
social_df = pd.merge(social_df, 구전체면적, on='자치구', how='left')

# 자치구별로 총생활인구수와 구난시설 수를 병합
social_df = pd.merge(social_df, 구난시설, on='자치구', how='left')

# 구난시설 비율 계산 (면적 대비 구난시설 개수)
social_df['구난시설 비율 (시설/면적)'] = social_df['구난시설'] / social_df['구전체면적']

# 필요한 데이터프레임 결합
social_data = pd.merge(cluster_counts, 종합_인구밀도, on='자치구', how='left')
social_data = pd.merge(social_data, social_df, on='자치구', how='left')

# 군집별 자치구 데이터 출력 함수
def print_cluster_details(social_data, cluster_number):
    cluster_data = social_data[social_data['Cluster'] == cluster_number]
    print(f"\nCluster {cluster_number} - 자치구별 소셜 데이터:")
    print(cluster_data[['자치구', '인구밀도 (명/㎢)', '노인 인구 비율 (%)', '장애인 인구 비율 (%)', '기초생활수급자_인구 비율 (%)', '구난시설 비율 (시설/면적)']])

# 예를 들어 군집 0의 데이터를 보고 싶다면
print_cluster_details(social_data, 0)

# 자치구별 info 정보 시각화

In [ ]:
import pandas as pd
from jinja2 import Template
import os

# HTML 템플릿 정의
html_template = """
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{{ 자치구 }} 정보</title>
    <style>
        body {
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f4;
            display: flex;
            flex-direction: column;
        }
        .sidebar {
            width: 300px;
            padding: 15px;
            background-color: #fff;
            border-right: 1px solid #ddd;
            box-shadow: 0 0 8px rgba(0, 0, 0, 0.1);
            overflow-y: auto;
            height: 100vh;
        }
        h1 {
            font-size: 1.6em;
            margin-bottom: 15px;
            color: #333;
        }
        .info {
            margin-bottom: 15px;
            font-size: 0.9em;
            color: #555;
        }
        .summary {
            font-size: 1em;
            color: #333;
            margin-bottom: 20px;
            border-top: 2px solid #ddd;
            padding-top: 10px;
            line-height: 1.6;
        }
        .metric {
            margin-bottom: 20px;
            padding: 12px;
            border-radius: 6px;
            background-color: #fff;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
        .metric h3 {
            margin: 0 0 8px;
            font-size: 1.2em;
            color: #444;
        }
        .metric p {
            margin: 0;
            font-size: 0.9em;
            color: #666;
            line-height: 1.6;
        }
        .highlight {
            color: #d9534f;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <div class="sidebar">
        <h1>{{ 자치구 }}의 주요 정보</h1>

        <!-- 구난시설 비율 설명 -->
        <div class="summary">
            <h2>구난시설 비율</h2>
            <p>구난시설 비율은 해당 지역의 구난시설이 면적에 비해 얼마나 배치되어 있는지를 나타냅니다.</p>
            <p>높은 비율은 구난시설이 잘 배치되어 있어 비상 상황에 대응하기 좋다는 것을 의미합니다.</p>
        </div>

        <!-- 구난시설 비율 카드 -->
        <div class="metric">
            <h3>구난시설 비율</h3>
            <p>구난시설 비율: <span class="highlight">{{ 구난시설_비율 }}%</span> (시설/면적)</p>
        </div>

        <!-- 불투수율 설명 -->
        <div class="summary">
            <h2>불투수율</h2>
            <p>불투수율은 비가 올 때 물이 지면에 스며드는 비율을 나타냅니다.</p>
            <p>높은 불투수율은 물이 지면에 잘 스며들지 않고 도로와 건물 주변에 고일 가능성이 높다는 것을 의미합니다.</p>
        </div>

        <!-- 불투수율 카드 -->
        <div class="metric">
            <h3>불투수율</h3>
            <p><strong>{{ 불투수율 }}%</strong></p>
            {% if 불투수율 > 서울평균_불투수율 %}
            <p>불투수율이 서울시 평균인 {{ 서울평균_불투수율 }}%보다 높습니다.</p>
            <p>물이가 잘 스며들지 않아 침수 위험이 클 수 있습니다.</p>
            {% else %}
            <p>불투수율이 서울시 평균인 {{ 서울평균_불투수율 }}%과 비슷하거나 낮습니다.</p>
            <p>침수 위험이 비교적 낮습니다.</p>
            {% endif %}
            {% if 불투수율 > 70 %}
            <p class="highlight">경고: 불투수율이 매우 높습니다. 침수에 대비하세요.</p>
            {% endif %}
        </div>

        <!-- 녹지율 설명 -->
        <div class="summary">
            <h2>녹지율</h2>
            <p>녹지율은 지역 내 녹지 면적의 비율을 나타냅니다.</p>
            <p>높은 녹지율은 식물과 나무가 많아 비가 올 때 빗물이 잘 흡수될 수 있음을 의미합니다.</p>
        </div>

        <!-- 녹지율 카드 -->
        <div class="metric">
            <h3>녹지율</h3>
            <p><strong>{{ 녹지율 }}%</strong></p>
            {% if 녹지율 < 서울평균_녹지율 %}
            <p>녹지율이 서울시 평균인 {{ 서울평균_녹지율 }}%보다 낮습니다.</p>
            <p>식물과 나무가 적어 빗물이 잘 흡수되지 않을 수 있습니다.</p>
            {% else %}
            <p>녹지율이 서울시 평균인 {{ 서울평균_녹지율 }}%과 비슷하거나 높습니다.</p>
            <p>많은 녹지 덕분에 빗물이 잘 흡수됩니다.</p>
            {% endif %}
        </div>
    </div>
</body>
</html>
"""

# HTML 파일로 저장할 디렉토리 생성
output_dir = 'district_info_html'
os.makedirs(output_dir, exist_ok=True)
불투수 = pd.read_csv('./불투수율.csv')
녹지 = pd.read_csv('./녹지.csv')

# 자치구별로 HTML 파일 생성
for index, row in social_data.iterrows():
    template = Template(html_template)
    # 소수점으로 된 비율 값을 퍼센트 값으로 변환
    구난시설_비율 = row['구난시설 비율 (시설/면적)'] * 100  # 퍼센트로 변환
    if not 불투수[불투수['자치구'] == row['자치구']].empty:
        불투수율 = round(불투수.loc[불투수['자치구'] == row['자치구'], '불투수면적 비율'].values[0],2)
    else:
        불투수율 = 0  # 기본값 또는 None

    # 녹지율 값 가져오기
    if not 녹지[녹지['자치구'] == row['자치구']].empty:
        녹지율 = round(녹지.loc[녹지['자치구'] == row['자치구'], '녹지율'].values[0],2)
    else:
        녹지율 = 0  # 기본값 또는 None
    서울평균_불투수율 = round(불투수['불투수면적 비율'].mean(),2) 
    서울평균_녹지율 = round(녹지['녹지율'].mean(),2)

    html_content = template.render(
        자치구=row['자치구'],
        구난시설_비율=f"{구난시설_비율:.1f}",
        불투수율=불투수율,
        녹지율=녹지율,
        서울평균_불투수율=서울평균_불투수율,
        서울평균_녹지율=서울평균_녹지율
    )

    # HTML 파일 경로 설정
    file_path = os.path.join(output_dir, f"{row['자치구']}_info.html")

    # HTML 파일로 저장
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(html_content)

print("모든 자치구에 대한 HTML 파일이 성공적으로 생성되었습니다.")


# 서울 info 시각화

In [ ]:
import pandas as pd
from jinja2 import Template
import os

# HTML 템플릿 정의
html_template = """
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{{ 자치구 }} 정보</title>
    <style>
        body {
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f4;
            display: flex;
            flex-direction: column;
        }
        .sidebar {
            width: 300px;
            padding: 15px;
            background-color: #fff;
            border-right: 1px solid #ddd;
            box-shadow: 0 0 8px rgba(0, 0, 0, 0.1);
            overflow-y: auto;
            height: 100vh;
        }
        h1 {
            font-size: 1.6em;
            margin-bottom: 15px;
            color: #333;
        }
        .info {
            margin-bottom: 15px;
            font-size: 0.9em;
            color: #555;
        }
        .summary {
            font-size: 1em;
            color: #333;
            margin-bottom: 20px;
            border-top: 2px solid #ddd;
            padding-top: 10px;
            line-height: 1.6;
        }
        .metric {
            margin-bottom: 20px;
            padding: 12px;
            border-radius: 6px;
            background-color: #fff;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
        .metric h3 {
            margin: 0 0 8px;
            font-size: 1.2em;
            color: #444;
        }
        .metric p {
            margin: 0;
            font-size: 0.9em;
            color: #666;
            line-height: 1.6;
        }
        .highlight {
            color: #d9534f;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <div class="sidebar">
        <h1>서울의 주요 정보</h1>

        <!-- 구난시설 비율 설명 -->
        <div class="summary">
            <h2>구난시설 비율</h2>
            <p>구난시설 비율은 해당 지역의 구난시설이 면적에 비해 얼마나 배치되어 있는지를 나타냅니다.</p>
            <p>높은 비율은 구난시설이 잘 배치되어 있어 비상 상황에 대응하기 좋다는 것을 의미합니다.</p>
        </div>

        <!-- 구난시설 비율 카드 -->
        <div class="metric">
            <h3>구난시설 비율</h3>
            <p>구난시설 비율: <span class="highlight">{{ 구난시설_비율 }}%</span> (시설/면적)</p>
        </div>

        <!-- 불투수율 설명 -->
        <div class="summary">
            <h2>불투수율</h2>
            <p>불투수율은 비가 올 때 물이 지면에 스며드는 비율을 나타냅니다.</p>
            <p>높은 불투수율은 물이 지면에 잘 스며들지 않고 도로와 건물 주변에 고일 가능성이 높다는 것을 의미합니다.</p>
        </div>

        <!-- 불투수율 카드 -->
        <div class="metric">
            <h3>불투수율</h3>
            <p>서울시 평균 불투수율은 {{ 서울평균_불투수율 }}%입니다.</p>
        </div>

        <!-- 녹지율 설명 -->
        <div class="summary">
            <h2>녹지율</h2>
            <p>녹지율은 지역 내 녹지 면적의 비율을 나타냅니다.</p>
            <p>높은 녹지율은 식물과 나무가 많아 비가 올 때 빗물이 잘 흡수될 수 있음을 의미합니다.</p>
        </div>

        <!-- 녹지율 카드 -->
        <div class="metric">
            <h3>녹지율</h3>
            <p>서울시 평균 녹지율은 {{ 서울평균_녹지율 }}%입니다.</p>

        </div>
    </div>
</body>
</html>
"""

# HTML 파일로 저장할 디렉토리 생성
output_dir = 'district_info_html'
os.makedirs(output_dir, exist_ok=True)
불투수 = pd.read_csv('./불투수율.csv')
녹지 = pd.read_csv('./녹지.csv')


template = Template(html_template)
# 소수점으로 된 비율 값을 퍼센트 값으로 변환
구난시설_비율 = row['구난시설 비율 (시설/면적)'] * 100  # 퍼센트로 변환
서울평균_불투수율 = round(불투수['불투수면적 비율'].mean(),2) 
서울평균_녹지율 = round(녹지['녹지율'].mean(),2)

html_content = template.render(
    구난시설_비율=f"{구난시설_비율:.1f}",
    서울평균_불투수율=서울평균_불투수율,
    서울평균_녹지율=서울평균_녹지율
)

# HTML 파일 경로 설정
file_path = os.path.join(output_dir, f"서울_info.html")

# HTML 파일로 저장
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(html_content)

print("모든 자치구에 대한 HTML 파일이 성공적으로 생성되었습니다.")